# Spreadsheets to HTML using pandas
This notebook fetches Google Spreadsheets to fill HTML templates.

The goal is to allow non-coders to edit the content, but also to be able to easily handcode some parts of the page for the fine-tuning.

At first, this was a quick solution coded in 5 hours. It is not an example of good coding practice…

In [1]:
import pandas as pd
from re import findall
import urllib.parse # to encode the URL
from datetime import datetime
import define # we store the FTP access in this file


BASEURL = 'https://tdg.ch/extern/interactive_wch/tdg/womeninaction/'
BASEURL_VQH = 'https://www.24heures.ch/extern/interactive_wch/womeninaction/'

LOCAL_FOLDER = '/Users/paul/Sites/womeninaction/'
LOCAL_FOLDER_VQH = '/Users/paul/Sites/womeninaction/vqh'

story_limit = 5
week = 4
week_url = "{}semaine{}.html".format(BASEURL, week)
week_titles = [None, 'Education', 'Economie', 'Santé', 'Société']
week_visuals = [None, 'Visuel-Sem1.jpg', 'Visuel-Sem2.jpg', 'Visuel-Sem3.jpg', 'Visuel-Sem4.jpg']
video_replacements = {
    'pWFxc1LHZeQ': '4Q_B-jhTUgA', # week 2
    '5K4i-9X8HwQ': '3cuOulVH1xI', # week 3
    '-rr9gvLef7U': 'SSlsUFPd7bU', # week 4
    '-NUI1Hl42PM': 'byuxHL01mvI', # also week 4    
}

In [2]:
# the URL of each Google Sheet
files = {
    1: [
        'https://docs.google.com/a/tdg.ch/spreadsheets/d/13rPxzFdu7ScFINTcCc_lHbd1_ep7Nt7fX6bV45TAYAs/export?format=csv&id=13rPxzFdu7ScFINTcCc_lHbd1_ep7Nt7fX6bV45TAYAs&gid=0',
        'https://docs.google.com/a/tdg.ch/spreadsheets/d/13rPxzFdu7ScFINTcCc_lHbd1_ep7Nt7fX6bV45TAYAs/export?format=csv&id=13rPxzFdu7ScFINTcCc_lHbd1_ep7Nt7fX6bV45TAYAs&gid=199185693',
        'https://docs.google.com/a/tdg.ch/spreadsheets/d/13rPxzFdu7ScFINTcCc_lHbd1_ep7Nt7fX6bV45TAYAs/export?format=csv&id=13rPxzFdu7ScFINTcCc_lHbd1_ep7Nt7fX6bV45TAYAs&gid=2058868552'
    ],
    2: [
        'https://docs.google.com/spreadsheets/d/1ci_Lo006eeY8qQcimKisn6D0ru-aXFCy7GiPSLQZ4AE/export?format=csv&id=1ci_Lo006eeY8qQcimKisn6D0ru-aXFCy7GiPSLQZ4AE&gid=0',
        'https://docs.google.com/spreadsheets/d/1ci_Lo006eeY8qQcimKisn6D0ru-aXFCy7GiPSLQZ4AE/export?format=csv&id=1ci_Lo006eeY8qQcimKisn6D0ru-aXFCy7GiPSLQZ4AE&gid=199185693',
        'https://docs.google.com/spreadsheets/d/1ci_Lo006eeY8qQcimKisn6D0ru-aXFCy7GiPSLQZ4AE/export?format=csv&id=1ci_Lo006eeY8qQcimKisn6D0ru-aXFCy7GiPSLQZ4AE&gid=2058868552'
    ],
    3: [
        'https://docs.google.com/a/tdg.ch/spreadsheets/d/14t2FUu-7KMmHtjdBadNAF2YjlMwhxYd57FMo-HVRIOM/export?format=csv&id=14t2FUu-7KMmHtjdBadNAF2YjlMwhxYd57FMo-HVRIOM&gid=0',
        'https://docs.google.com/a/tdg.ch/spreadsheets/d/14t2FUu-7KMmHtjdBadNAF2YjlMwhxYd57FMo-HVRIOM/export?format=csv&id=14t2FUu-7KMmHtjdBadNAF2YjlMwhxYd57FMo-HVRIOM&gid=199185693',
        'https://docs.google.com/a/tdg.ch/spreadsheets/d/14t2FUu-7KMmHtjdBadNAF2YjlMwhxYd57FMo-HVRIOM/export?format=csv&id=14t2FUu-7KMmHtjdBadNAF2YjlMwhxYd57FMo-HVRIOM&gid=2058868552'
    ],
    4: [
        'https://docs.google.com/a/tdg.ch/spreadsheets/d/1-FbN9WxYD1d9ZB0NWL_u2MQ6SVmfkKuSrAsNrmORihk/export?format=csv&id=1-FbN9WxYD1d9ZB0NWL_u2MQ6SVmfkKuSrAsNrmORihk&gid=0',
        'https://docs.google.com/a/tdg.ch/spreadsheets/d/1-FbN9WxYD1d9ZB0NWL_u2MQ6SVmfkKuSrAsNrmORihk/export?format=csv&id=1-FbN9WxYD1d9ZB0NWL_u2MQ6SVmfkKuSrAsNrmORihk&gid=199185693',
        'https://docs.google.com/a/tdg.ch/spreadsheets/d/1-FbN9WxYD1d9ZB0NWL_u2MQ6SVmfkKuSrAsNrmORihk/export?format=csv&id=1-FbN9WxYD1d9ZB0NWL_u2MQ6SVmfkKuSrAsNrmORihk&gid=2058868552'
    ]
}

In [3]:
df = pd.read_csv(files[week][0])
dft = pd.read_csv(files[week][1])
dfi = pd.read_csv(files[week][2])

In [4]:
df = df.merge(dft, on='No')
df.head(5)

,No,Titre,Date,Auteur(s),Média,En savoir plus,URL image,Légende image,Crédit image,Chapeau,Corps du texte
0,1,"Au Nigéria, des femmes donnent le signal du ch...",4 décembre 2017,Praise Olowe,The Nation (Nigéria),http://wisenigeria.org/,https://files.newsnetz.ch/upload//1/5/151081.jpg,De nombreuses Nigérianes sont astreintes à des...,Olanike Oluboji,Une organisation nigériane à but non lucratif ...,"Dans le Nigéria rural, cuisiner peut tuer. Sel..."
1,2,"Au Sénégal, coup d’envoi d’une nouvelle ère po...",5 décembre 2017,Idrissa Sané,Le Soleil (Sénégal),http://www.ladiesturn.org/LadiesTurn_Francais/...,https://files.newsnetz.ch/upload//1/5/151769.jpg,"La présidente de Ladies' Turn, Seyni Ndir Seck...",Ladies' Turn,L'association Ladies' Turn promeut le sport fé...,"Dans l’atmosphère frénétique du nord de Dakar,..."
2,3,"À Saint-Denis, une Maison des femmes au secour...",6 décembre 2017,Stéphane Kovacs,Le Figaro (France),https://www.lamaisondesfemmes.fr/,https://files.newsnetz.ch/upload//1/5/151999.jpg,La gynécologue Ghada Hatem a fondé la Maison d...,AFP,Dans cette ville où l’accès aux soins passe es...,"«Entre ces murs, il n’y aura pas de tabou», pr..."
3,4,Du théâtre et des jeux pour briser la violence...,7 décembre 2017,Soumya Pillai,The Hindu (Inde),https://www.inbreakthrough.tv/,https://files.newsnetz.ch/upload//1/5/152259.jpg,Les pièces de théâtre présentées en plein air ...,Breakthrough India,L'ONG Breakthrough India aide les femmes à com...,"Sunaina, 14 ans, vit dans un village du nord d..."
4,5,"Au Burkina Faso, la loi sur les quotas de femm...",8 décembre 2017,Sandrine Sawadogo,L'Economiste du Faso,http://lefaso.net/spip.php?article77379,https://www.tdg.ch/extern/interactive_wch/tdg/...,Le parlement burkinabé en 2008. Huit ans après...,AFP,"Depuis le 16 avril 2009, 30% au moins des cand...","Depuis le 16 avril 2009, 30% au moins des cand..."


In [5]:
teaser = '''        <div class="col-md-8 col-md-offset-2 col-xs-12 cell" onclick="$('#article{storyid}').modal();" data-modal="article{storyid}">
        <h4>{h4}</h4>
        <h2>{h2}</h2>
        <img class="teaser" src="{img}" />
        <p class="lead">{lead}</p>
      </div>
'''

In [6]:
img_and_caption = '''            <img class="stretch" src="{img}" />
            <p class="legende">{caption} <span class="credit">{credit}</span></p>
'''

In [7]:
def getSocial(_id, _titre):
    _url = week_url + '?article=' + str(_id)
    if not _id:
        _url = week_url
    _titre = urllib.parse.quote_plus(_titre)
    url_title = _url + '%20' + _titre
    
    social = '''       <a href="https://twitter.com/intent/tweet?text={url_title}&amp;hashtags=WomenInAction,ImpactWomen" title="Partager sur Twitter" target="_blank" class="btn btn-twitter"><i class="fa fa-twitter"></i></a>
       <a href="https://www.facebook.com/sharer/sharer.php?u={url}" title="Partager sur Facebook" target="_blank" class="btn btn-facebook"><i class="fa fa-facebook"></i></a>
       <a href="http://www.linkedin.com/shareArticle?mini=true&amp;url={url}&amp;title={title}" title="Partager sur LinkedIn" target="_blank" class="btn btn-linkedin"><i class="fa fa-linkedin"></i></a>'''
    return social.format(url=_url, url_title=url_title, title=_titre)

In [8]:
def shortenText(text):
    text = text.replace('http://', '').replace('https://', '').replace('www.', '')
    parts = [i for i in text.split('/') if i != '']
    if len(parts) > 1:
        return parts[0] + '/' + '…'
    else:
        return text.strip('/')
def getMoreLink(_link):
    if _link != _link or _link == '':
        return ''
    else:
        return '<p class="more_link">En savoir plus: <a target="_blank" href="{}">{}</a></p>'.format(_link, shortenText(_link))

In [9]:
dfi

,No,No image,Id (auto),Légende,Crédit
0,4,1,4-1,Une session de sensibilisation des communautés...,Breakthrough India
1,4,2,4-2,NaN,Breakthrough India
2,4,5,4-5,Le programme «Taaron Ki Toli» promeut l’égalit...,Breakthrough India


In [10]:
images_corps = {}
for i, row in dfi.iterrows():
    url_img = '_img/chap{}/modal{}/photo{}.jpg'.format(week, row['No'], row['No image'])
    images_corps[row['Id (auto)']] = img_and_caption.format(img=url_img, caption=row['Légende'], credit=row['Crédit'])
images_corps

{'4-1': '            <img class="stretch" src="_img/chap4/modal4/photo1.jpg" />\n            <p class="legende">Une session de sensibilisation des communautés et des travailleurs de santé organisée par Breakthrough dans le cadre de la campagne «Mission 1000». <span class="credit">Breakthrough India</span></p>\n',
 '4-2': '            <img class="stretch" src="_img/chap4/modal4/photo2.jpg" />\n            <p class="legende">nan <span class="credit">Breakthrough India</span></p>\n',
 '4-5': '            <img class="stretch" src="_img/chap4/modal4/photo5.jpg" />\n            <p class="legende">Le programme «Taaron Ki Toli» promeut l’égalité hommes-femmes dans les écoles. <span class="credit">Breakthrough India</span></p>\n'}

In [11]:
with open('modal_template.html', 'r') as f: story_template = f.read()

### Uncomment the following to get a quick list of modal template variables

In [12]:
# print(*findall('{(.*?)}', story_template), sep="=row['xxx'], ")

In [13]:
def addParagraphs(rawtext):
    rawtext = rawtext.replace('« ', '«').replace(' »', '')
    p_list = rawtext.split('\n')
    for p in p_list:
        if p.find('.') < 0:
            p = '<b>' + p + '</b>'
    return "<p>" + "</p>\n<p>".join(p_list) + "</p>"

In [14]:
teasers = ''
modals = ''
nextSubject = ''
for i, row in df[:story_limit].iterrows():
    if not isinstance(row['Corps du texte'], str):
        nextSubject = row['Titre']
        break
    
    body = row['Corps du texte'].strip()
    if body.find('<p>') < 0:
        body = addParagraphs(body)
    
    social = getSocial(row['No'], row['Titre'])
    teasers = teaser.format(storyid=row['No'], h4=row['Date'], h2=row['Titre'],
                            img=row['URL image'], lead=row['Chapeau']) + teasers
    
    modals += story_template.format(storyid=row['No'], title=row['Titre'], lead=row['Chapeau'],
                                    social=getSocial(row['No'], row['Titre']),
                                    more_link=getMoreLink(row['En savoir plus']),
                                    img=row['URL image'], legend=row['Légende image'],
                                    credit=row['Crédit image'], author=row['Auteur(s)'],
                                    media=row['Média'], body=body)

In [15]:
with open('template_week_{}.html'.format(week), 'r', encoding="utf-8") as f: template = f.read()
template = template.replace('[TEASERS]', teasers).replace('[MODALS]', modals)

In [16]:
for i in findall('(PHOTO.*?-\d)', template):
    print(i)

PHOTO 4-1
PHOTO 4-5


In [17]:
images_corps

{'4-1': '            <img class="stretch" src="_img/chap4/modal4/photo1.jpg" />\n            <p class="legende">Une session de sensibilisation des communautés et des travailleurs de santé organisée par Breakthrough dans le cadre de la campagne «Mission 1000». <span class="credit">Breakthrough India</span></p>\n',
 '4-2': '            <img class="stretch" src="_img/chap4/modal4/photo2.jpg" />\n            <p class="legende">nan <span class="credit">Breakthrough India</span></p>\n',
 '4-5': '            <img class="stretch" src="_img/chap4/modal4/photo5.jpg" />\n            <p class="legende">Le programme «Taaron Ki Toli» promeut l’égalité hommes-femmes dans les écoles. <span class="credit">Breakthrough India</span></p>\n'}

In [18]:
for i in findall('(PHOTO.*?-\d)', template):
    targetImage = images_corps[i.split(' ')[-1]]
    print(i, 'remplacé par', targetImage)
    print('---')
    template = template.replace(i, targetImage)

PHOTO 4-1 remplacé par             <img class="stretch" src="_img/chap4/modal4/photo1.jpg" />
            <p class="legende">Une session de sensibilisation des communautés et des travailleurs de santé organisée par Breakthrough dans le cadre de la campagne «Mission 1000». <span class="credit">Breakthrough India</span></p>

---
PHOTO 4-5 remplacé par             <img class="stretch" src="_img/chap4/modal4/photo5.jpg" />
            <p class="legende">Le programme «Taaron Ki Toli» promeut l’égalité hommes-femmes dans les écoles. <span class="credit">Breakthrough India</span></p>

---


In [19]:
template = template.replace('NOM_CHAPITRE', week_titles[week])
template = template.replace('VISUEL_SEMAINE', week_visuals[week])

In [20]:
filename = 'semaine{}.html'.format(week)
tempFilename = 'semaine{}-test.html'.format(week)

In [21]:
with open('{}{}'.format(LOCAL_FOLDER, filename), 'w', encoding="utf-8") as outputfile:
    outputfile.write(template)

In [22]:
if week == 2:
    !open -a "Google Chrome" '/Users/paul/Sites/womeninaction/semaine2.html'
elif week == 3:
    !open -a "Google Chrome" '/Users/paul/Sites/womeninaction/semaine3.html'
elif week == 4:
    !open -a "Google Chrome" '/Users/paul/Sites/womeninaction/semaine4.html'

In [23]:
with open(filename, 'w', encoding="utf-8") as outputfile:
    outputfile.write(template)

### Conversion pour VQH

In [24]:
with open('{}'.format(filename), 'r', encoding="utf-8") as indexfile:
    tempContents = indexfile.read()

In [25]:
tempContents = tempContents.replace(BASEURL, BASEURL_VQH).replace('tdg.ch', '24heures.ch')
tempContents = tempContents.replace('</title>', '</title>\n<base href="https://www.24heures.ch/extern/interactive_wch/tdg/womeninaction/" />')

for key, value in video_replacements.items():
    tempContents = tempContents.replace(key, value)

In [26]:
with open('vqh/{}'.format(filename), 'w', encoding="utf-8") as outputfile:
    outputfile.write(tempContents)

# FTP

In [27]:
from ftplib import FTP
ftp = FTP(define.host, define.user, define.password)  

In [28]:
ftp.cwd('/tdg/womeninaction')

'250 CWD command successful'

In [29]:
temp_filename = 'semaine{}-test.html'.format(week)
ftp.storlines("STOR " + temp_filename, open('{}'.format(filename), 'rb'))


'226 Transfer complete'

## Put into prod tdg.ch

In [30]:
goForward = input()
if goForward != '':
    raise ValueError('Script interrompu')

In [31]:
targetName = datetime.now().strftime("semaine{}_%m-%d_%Hh%M") + '.html'
targetName = targetName.format(week)
targetName

'semaine4_12-08_17h03.html'

In [32]:
filename

'semaine4.html'

In [33]:
#FTP.rename?
ftp.rename(filename, targetName)
ftp.rename(tempFilename, filename)

'250 Rename successful'

## Put into prod 24heures.ch

In [35]:
ftp.storlines("STOR " + temp_filename, open('vqh/{}'.format(filename), 'rb'))



'226 Transfer complete'

In [36]:
#FTP.rename?
#ftp.rename(filename, targetName)
ftp.rename(tempFilename, filename)

'250 Rename successful'

In [37]:
print(BASEURL + 'semaine{}.html?article={}'.format(week, story_limit))
print(BASEURL_VQH + 'semaine{}.html?article={}'.format(week, story_limit))

https://tdg.ch/extern/interactive_wch/tdg/womeninaction/semaine4.html?article=5
https://www.24heures.ch/extern/interactive_wch/womeninaction/semaine4.html?article=5


In [38]:
print(week_url + '?article=' + str(story_limit))

https://tdg.ch/extern/interactive_wch/tdg/womeninaction/semaine4.html?article=5


### These commands will only work on a Mac and some other unix-based system

In [39]:
!open -a "Google Chrome" 'https://www.tdg.ch/extern/interactive_wch/tdg/womeninaction/semaine4.html'

In [40]:
!open -a "Google Chrome" 'https://www.24heures.ch/extern/interactive_wch/womeninaction/semaine4.html'